In [1]:
import torch
from torch.utils.data import Dataset
import cv2
import numpy as np
import random
from torchvision.transforms import RandomRotation, ToPILImage, ToTensor, ColorJitter
import torchvision.transforms.functional as TF
#from yolo.detect import babybox
#from yolo.models import *
#from yolo.utils import *
from utils import img2uint8, pad_to_square
tr = torch

In [2]:
import glob
from sklearn.model_selection import KFold
from PIL import Image

In [3]:
def load_image(path): #Input:CH,H,W  Output:H,W,CH
        image_pil = Image.open(str(path))
        img = np.asarray(image_pil)
        #img=img.transpose((1, 2, 0))  
        return img

In [4]:
class DatasetDeepPhys(Dataset):
    """
        Dataset class for training network.
    """
    


    def __init__(self, img_paths: str, device, start=None, end=None, shift=0, crop=True, augment=False,limit=False):

        # param img_paths: path to the images
        # param device: GPU or CPU
        
        from utils import ReferenceProcessor #Funcion para calcular la derivada de la señal PPG
        
        # 1.- Parametros requeridos 
        self.H = 36
        self.W = 36
        self.C = 3
        self.augment = augment
        self.device = device
        self.crop = crop
        self.img_paths = img_paths
        self.limit=limit
        # -----------------------------
        # Augmentation variables
        # ----------------------------
        self.flip_p = None
        self.rot = None
        self.color_transform = None
        
        
        # 2.- Carga del dataset a partir de un archivo hdf5
        """
        self.db_path = path
        db = h5py.File(path, 'r')
        frames = db['frames']
        db_labels = db['references']
        """
        
        # Esta parte no es necesaria por que usamos un video donde ya esta casi delimitado el rostro
        """"
        # check if there is bbox
        keys = db.keys()
        print(keys)
        self.is_bbox = 'bbox' in keys
        if self.is_bbox:
            print('\nBounding boxes found in database, lets use them instead of YOLO!')
            

        # -----------------------------
        # Init baby cropper
        # -----------------------------
        if crop and not self.is_bbox:
            model_def = 'yolo/config/yolov3-custom.cfg'
            weight_path = 'yolo/weights/yolov3_ckpt_42.pth'

            self.yolo = Darknet(model_def).to(device)
            self.yolo.load_state_dict(torch.load(weight_path))
            self.yolo.eval()
            print("YOLO network is initialized and ready to work!")
        """
        
        """
        # 3.- Carga de la señal PPG y calculo de su derivada (esta parte problemas)
        # Create derivated ppg label
        ppg_label = db_labels['PPGSignal']
        refproc = ReferenceProcessor(ppg_label[shift:])
        refproc.calculate()
        self.label = refproc.training_label
        
        

        (self.n, H, W, C) = frames.shape

        if start is not None:
            self.n = end - start
            self.begin = start
        else:
            self.begin = 0

        print(f'\nNumber of images in the dataset: {self.n}')
        print(f'Size of an image: {H} x {W} x {C}')

        # self.num_samples = self.n - 1 - shift
        # To be in line with physnet
        
        
        
        # NO entiendo bien
        tmp = ((self.n - 64) // 128) 
        self.num_samples = tmp * 128 - 1 - shift
        db.close()
        """

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        # Data augmentation en caso sea necesario
        if self.augment:
            # Set up the same image transforms for the chunk
            self.flip_p = random.random()
            self.hflip_p = random.random()
            # self.rot = RandomRotation.get_params((0, 90))
            self.color_transform = ColorJitter.get_params(brightness=(0.3, 1.5),
                                                          contrast=(0.8, 1.2),
                                                          saturation=(0.8, 1.2),
                                                          hue=(0, 0))

            
        # Target construido a partir de la señal PPG derivada
        # Construct target signals
        #arget = tr.tensor(self.label[idx])
        
        

        # Construct networks input
        A = tr.empty(self.C, self.H, self.W, dtype=tr.float)
        M = tr.empty(self.C, self.H, self.W, dtype=tr.float)
        
        
        #4.1.- Carga del dataset a partir de hdf5 files
        """
        with h5py.File(self.db_path, 'r') as db:
            frames = db['frames']
            img1 = frames[idx, :, :, :]
            img2 = frames[idx + 1, :, :, :]

            # ----------------------------
            # Crop baby with yolo
            # ----------------------------
            if self.crop and self.is_bbox:
                bbox = db['bbox'][idx, :]
                y1, y2, x1, x2 = bbox[0], bbox[1], bbox[2], bbox[3]

                # check to be inside image size
                if y2 > img1.shape[0]:
                    y2 = img1.shape[0]
                if x2 > img1.shape[1]:
                    x2 = img1.shape[1]
                if y1 < 0:
                    y1 = 0
                if x1 < 0:
                    x1 = 0
                # check validity
                if y2-y1 < 1 or x2-x1 < 1:
                    y1 = x1 = 0
                    y2, x2 = img1.shape[:2]

                img1 = img1[y1:y2, x1:x2, :]
                img2 = img2[y1:y2, x1:x2, :]
            elif self.crop and not self.is_bbox:
                x1, y1, x2, y2 = babybox(self.yolo, img1, self.device)
                img1 = img1[y1:y2, x1:x2, :]
                img2 = img2[y1:y2, x1:x2, :]     
        """      
        #4.2- Carga del dataset a partir de img files (mantener transformaciones)
        
        if self.limit is None:
            img_file_name_1 = self.img_paths[idx]
            img_file_name_2 = self.img_paths[idx+1]
        else:
            img_file_name_1 = np.random.choice(self.img_paths)
            img_file_name_2 = np.random.choice(self.img_paths)
        
        img1 = load_image(img_file_name_1)
        img2 = load_image(img_file_name_2)
        

        # Downsample image
        try:
            img1 = cv2.resize(img1, (self.H, self.W), interpolation=cv2.INTER_CUBIC)
            img2 = cv2.resize(img2, (self.H, self.W), interpolation=cv2.INTER_CUBIC)
        except:
            print('\n--------- ERROR! -----------\nUsual cv empty error')
            print(f'Shape of img1: {img1.shape}; Shape of im2: {img2.shape}')
            print(f'bbox: {bbox}')
            print(f'This is at idx: {idx}')
            exit(666)

        if self.augment:
            img1 = ToPILImage()(img1)
            img2 = ToPILImage()(img2)
            if self.flip_p > 0.5:
                img1 = TF.vflip(img1)
                img2 = TF.vflip(img2)
            if self.flip_p > 0.5:
                img1 = TF.hflip(img1)
                img2 = TF.hflip(img2)

            # img1 = TF.rotate(img1, self.rot)
            # img2 = TF.rotate(img2, self.rot)

            img1 = self.color_transform(img1)
            img2 = self.color_transform(img2)

            img1 = tr.from_numpy(np.array(img1).astype(np.float32))
            img2 = tr.from_numpy(np.array(img2).astype(np.float32))
            img1 = img1.permute(2, 0, 1)
            img2 = img2.permute(2, 0, 1)
        else:
            img1 = tr.from_numpy(img1.astype(np.float32))
            img2 = tr.from_numpy(img2.astype(np.float32))
            # Swap axes because  numpy image: H x W x C | torch image: C X H X W
            img1 = img1.permute(2, 0, 1)
            img2 = img2.permute(2, 0, 1)


    
        # 2.) construct the normalized frame difference for motion stream input
        M = tr.div(img2 - img1, img1 + img2 + 1)         # +1 for numerical stability
        # M = tr.sub(M, tr.mean(M, (1, 2)).view(3, 1, 1))  # spatial intensity norm for each channel

        A = img1/255.  # convert image to [0, 1]
        print(A.shape)
        A = tr.sub(A, tr.mean(A, (1, 2)).view(3, 1, 1))  # spatial intensity norm for each channel

        #sample = ((A, M), target)
        sample = (A,M)

        # Video shape: C x D x H X W
        return sample

In [5]:

data_path='C:/Users/Ramfis/Desktop/Universidad/dataset 2/VIPL-HR-V1/VIPL-HR-V1/data/p1/v1/source1/frames'
train_val_file_names=np.array(sorted(glob.glob(str(data_path)+ "/*.jpg")))
test_file_names =  np.array(sorted(glob.glob(str(data_path) + "/*.jpg")))

In [6]:
train_val_file_names

array(['C:/Users/Ramfis/Desktop/Universidad/dataset 2/VIPL-HR-V1/VIPL-HR-V1/data/p1/v1/source1/frames\\frame1.jpg',
       'C:/Users/Ramfis/Desktop/Universidad/dataset 2/VIPL-HR-V1/VIPL-HR-V1/data/p1/v1/source1/frames\\frame10.jpg',
       'C:/Users/Ramfis/Desktop/Universidad/dataset 2/VIPL-HR-V1/VIPL-HR-V1/data/p1/v1/source1/frames\\frame100.jpg',
       ...,
       'C:/Users/Ramfis/Desktop/Universidad/dataset 2/VIPL-HR-V1/VIPL-HR-V1/data/p1/v1/source1/frames\\frame997.jpg',
       'C:/Users/Ramfis/Desktop/Universidad/dataset 2/VIPL-HR-V1/VIPL-HR-V1/data/p1/v1/source1/frames\\frame998.jpg',
       'C:/Users/Ramfis/Desktop/Universidad/dataset 2/VIPL-HR-V1/VIPL-HR-V1/data/p1/v1/source1/frames\\frame999.jpg'],
      dtype='<U107')

In [7]:
def get_split_in(train_file_names, fold, num_splits=5):
 
    kf = KFold(n_splits=num_splits, random_state=2019,shuffle=True)
    #kf = KFold(n_splits=num_splits, random_state=20018)


    ids = list(kf.split(train_file_names))

    train_ids, val_ids = ids[fold]

    if fold == -1:
        return train_file_names, train_file_names
    else:
        return train_file_names[train_ids], train_file_names[val_ids]
    
def percent_split(train_val_100percent, percent = 1): 
    
    fpath_list = train_val_100percent


    dataset_size = len(fpath_list)
    indices = list(range(dataset_size))
    percent = int(np.floor(percent * dataset_size))
    if 1 :
        np.random.seed(2019)
        np.random.shuffle(indices)        

    extra_indices, train_indices_split = indices[percent:], indices[:percent]
    print(dataset_size,len(train_indices_split), len(extra_indices))
    
   
    return train_val_100percent[extra_indices],train_val_100percent[train_indices_split] #

def get_split_out(data_path,name_file, fold, num_splits=5):
    #data_path=Path(path)

    train_path = data_path / name_file / 'images'

    train_file_names = np.array(sorted(list(train_path.glob('*npy'))))

    kf = KFold(n_splits=num_splits, random_state=2019,shuffle=True)
    


    ids = list(kf.split(train_file_names))

    train_ids, val_ids = ids[fold]

    if fold == -1:
        return train_file_names, train_file_names
    else:
        return train_file_names[train_ids], train_file_names[val_ids]

In [8]:
train_file_names,val_file_names = get_split_in(train_val_file_names,-1)  

In [9]:
train_file_names

array(['C:/Users/Ramfis/Desktop/Universidad/dataset 2/VIPL-HR-V1/VIPL-HR-V1/data/p1/v1/source1/frames\\frame1.jpg',
       'C:/Users/Ramfis/Desktop/Universidad/dataset 2/VIPL-HR-V1/VIPL-HR-V1/data/p1/v1/source1/frames\\frame10.jpg',
       'C:/Users/Ramfis/Desktop/Universidad/dataset 2/VIPL-HR-V1/VIPL-HR-V1/data/p1/v1/source1/frames\\frame100.jpg',
       ...,
       'C:/Users/Ramfis/Desktop/Universidad/dataset 2/VIPL-HR-V1/VIPL-HR-V1/data/p1/v1/source1/frames\\frame997.jpg',
       'C:/Users/Ramfis/Desktop/Universidad/dataset 2/VIPL-HR-V1/VIPL-HR-V1/data/p1/v1/source1/frames\\frame998.jpg',
       'C:/Users/Ramfis/Desktop/Universidad/dataset 2/VIPL-HR-V1/VIPL-HR-V1/data/p1/v1/source1/frames\\frame999.jpg'],
      dtype='<U107')

In [10]:
def deepphys_dset_test(idx):
    # train on the GPU or on the CPU, if a GPU is not available
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    dset = DatasetDeepPhys(train_file_names, device)
    A, M = dset[idx]

    #print('target: ', point)

    A = A.permute(1, 2, 0).data.cpu().numpy()
    M = M.permute(1, 2, 0).data.cpu().numpy()

    cv2.namedWindow('A', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('A', 500, 500)
    A = convert2cvimshow(A)
    cv2.imshow('A', A)

    cv2.namedWindow('M', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('M', 500, 500)
    M = convert2cvimshow(M)
    cv2.imshow('M', M)

    while cv2.waitKey(1) & 0xFF != ord('q'):
        pass

    # Release everything if job is finished
    cv2.destroyAllWindows()

In [11]:
def convert2cvimshow(img):
    frame = img.copy()
    frame = frame - np.min(frame)
    frame = frame / np.max(frame)
    frame = (frame * 255).astype(np.uint8)
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    return frame

In [12]:
deepphys_dset_test(10)

torch.Size([3, 36, 36])
